In [1]:
import numpy as np
import pandas as pd
import random

In [2]:
test = pd.read_csv('./test.csv')
test_nolabel = pd.read_csv('./test_nolabel.csv')
train = pd.read_csv('./train.csv')
train_label = pd.read_csv('./train_label.csv')

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91531 entries, 0 to 91530
Data columns (total 33 columns):
ID                                91531 non-null int64
hotel                             91531 non-null object
is_canceled                       91531 non-null int64
lead_time                         91531 non-null int64
arrival_date_year                 91531 non-null int64
arrival_date_month                91531 non-null object
arrival_date_week_number          91531 non-null int64
arrival_date_day_of_month         91531 non-null int64
stays_in_weekend_nights           91531 non-null int64
stays_in_week_nights              91531 non-null int64
adults                            91531 non-null int64
children                          91527 non-null float64
babies                            91531 non-null int64
meal                              91531 non-null object
country                           91063 non-null object
market_segment                    91531 non-null object
dist

In [4]:
train.isna().sum()

ID                                    0
hotel                                 0
is_canceled                           0
lead_time                             0
arrival_date_year                     0
arrival_date_month                    0
arrival_date_week_number              0
arrival_date_day_of_month             0
stays_in_weekend_nights               0
stays_in_week_nights                  0
adults                                0
children                              4
babies                                0
meal                                  0
country                             468
market_segment                        0
distribution_channel                  0
is_repeated_guest                     0
previous_cancellations                0
previous_bookings_not_canceled        0
reserved_room_type                    0
assigned_room_type                    0
booking_changes                       0
deposit_type                          0
agent                             13217


In [5]:
df = train.drop(['ID', 'is_canceled', 'adr', 'reservation_status', 'reservation_status_date'], axis=1)

In [6]:
def preprocess(df):
    df['children'] = df['children'].fillna(-1)
    df['company'] = df['company'].fillna(-1).astype(object)
    df['agent'] = df['agent'].fillna(-1).astype(object)
    df['country'] = df['country'].fillna('NA')
    df['arrival_date_year'] = df['arrival_date_year'].astype(object)
    df['arrival_date_week_number'] = df['arrival_date_year'].astype(object)
    df['arrival_date_day_of_month'] = df['arrival_date_day_of_month'].astype(object)
    df['children'] = df['children'].astype(np.int64)
preprocess(df)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91531 entries, 0 to 91530
Data columns (total 28 columns):
hotel                             91531 non-null object
lead_time                         91531 non-null int64
arrival_date_year                 91531 non-null object
arrival_date_month                91531 non-null object
arrival_date_week_number          91531 non-null object
arrival_date_day_of_month         91531 non-null object
stays_in_weekend_nights           91531 non-null int64
stays_in_week_nights              91531 non-null int64
adults                            91531 non-null int64
children                          91531 non-null int64
babies                            91531 non-null int64
meal                              91531 non-null object
country                           91531 non-null object
market_segment                    91531 non-null object
distribution_channel              91531 non-null object
is_repeated_guest                 91531 non-null int64
pr

In [12]:
df_test = test.drop(['ID'], axis=1)
preprocess(df_test)
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27859 entries, 0 to 27858
Data columns (total 28 columns):
hotel                             27859 non-null object
lead_time                         27859 non-null int64
arrival_date_year                 27859 non-null object
arrival_date_month                27859 non-null object
arrival_date_week_number          27859 non-null object
arrival_date_day_of_month         27859 non-null object
stays_in_weekend_nights           27859 non-null int64
stays_in_week_nights              27859 non-null int64
adults                            27859 non-null int64
children                          27859 non-null int64
babies                            27859 non-null int64
meal                              27859 non-null object
country                           27859 non-null object
market_segment                    27859 non-null object
distribution_channel              27859 non-null object
is_repeated_guest                 27859 non-null int64
pr

In [8]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score, mean_squared_error

In [9]:
y = train['adr']
X_train, X_val, y_train, y_val = train_test_split(df, y, test_size=0.2, random_state = 50)

features_to_encode = df.columns[df.dtypes==object].tolist()
col_trans = make_column_transformer((OneHotEncoder(handle_unknown='ignore'), features_to_encode), remainder='passthrough')

In [10]:
regr = make_pipeline(col_trans, svm.SVR(kernel='linear', C=1, epsilon=0.1, gamma='auto'))

In [11]:
regr.fit(df, y)

print(mean_squared_error(y, regr.predict(df)))
y_pred = regr.predict(df_test)

print(y_pred)
adr_result = pd.DataFrame(data=y_pred)
adr_result.to_csv('adr.csv', index=False)

1197.0590473865068


NameError: name 'df_test' is not defined

In [13]:
y_pred = regr.predict(df_test)

print(y_pred)
adr_result = pd.DataFrame(data=y_pred)
adr_result.to_csv('adr.csv', index=False)

[112.41680009  61.7706872   75.74398907 ... 115.81718755  84.32828722
 133.04569924]


In [14]:
y = train['reservation_status']

In [15]:
clf = make_pipeline(col_trans, svm.SVC(kernel='linear', C=1, gamma='auto'))

In [16]:
clf.fit(df, y)

print(clf.score(df, y))
y_pred = clf.predict(df_test)

print(y_pred)
rs_result = pd.DataFrame(data=y_pred)
rs_result.to_csv('rs.csv', index=False)

0.8483683123750423
['Check-Out' 'Canceled' 'Check-Out' ... 'Check-Out' 'Check-Out'
 'Check-Out']


In [17]:
import math
import datetime

In [18]:
df = test
df['adr'] = adr_result
df['reservation_status'] = rs_result
df = df[df['reservation_status']!='Canceled']
df = df.drop(df[(df['reservation_status']=='No show') & (df['deposit_type']=='No Deposit')].index)
df['stays_in_nights'] = df['stays_in_weekend_nights'] + df['stays_in_week_nights']
df['label'] = df['adr']*df['stays_in_nights']
df['arrival_date']=pd.to_datetime(df['arrival_date_year'].astype(int).astype(str)  + df['arrival_date_month'] + df['arrival_date_day_of_month'].astype(int).astype(str),format='%Y%B%d')
result = df[['arrival_date', 'label']]
result = np.floor(result.groupby('arrival_date').sum()/10000)
result

,label
arrival_date,
2017-04-01,2.0
2017-04-02,3.0
2017-04-03,4.0
2017-04-04,1.0
2017-04-05,3.0
2017-04-06,2.0
2017-04-07,2.0
2017-04-08,3.0
2017-04-09,4.0


In [19]:
result['label'].to_csv ('result.csv', index = True, header=True)